# 1. Deutsch's algorithm

## 1a) Implement the algorithm by modifying the following cells

Below you will find the definition of the quantum querz gate $U_f$ for a secret function $f:\{0,1\}^n\rightarrow\{0,1\}$. Using Deutsch's algorithm, check whether secret_function is *constant* or *balanced* with query-complexity 1. 

In [3]:
from qiskit import QuantumCircuit
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram

In [4]:
def secret_function():
    f = QuantumCircuit(2)
    f.cx(0, 1)
    return f

We can check what the circuit looks like using the `draw` method.

In [ ]:
display(secret_function().draw(output='mpl'))

Now, you will create the actual quantum circuit for Deutsch's algorithm, using the query gate defined above.

Barriers are included to show the visual separation between the query gate implementation and the rest of the circuit.

In [12]:
def your_circuit(function: QuantumCircuit):
    # Compiles a circuit for use in Deutsch's algorithm.

    # this initializes a circuit with n+1 qubits and n classical bits
    n = function.num_qubits - 1
    qc = QuantumCircuit(n + 1, n)

    # remember that in Qiskit, all qubits start in the |0⟩ state

    # anything that happens before the query gate goes here

    qc.barrier()
    qc.compose(function, inplace=True)
    qc.barrier()

    # anything that happens after the query gate goes here

    # this examplary measurement measures the second qubit into the classical bit
    qc.measure(1, 0)

    return qc

In [ ]:
# check that your circuit is constructed correctly
display(your_circuit(secret_function()).draw(output='mpl'))

In [ ]:
def deutsch_algorithm(function: QuantumCircuit):
    # Determine if a one-bit function is constant or balanced.

    qc = your_circuit(function)

    # the following two lines extract the measurement result
    result = AerSimulator().run(qc, shots=1, memory=True).result()
    measurements = result.get_memory()

    if # write your condition here:
        return "constant"
    return "balanced"

In [ ]:
display(deutsch_algorithm(secret_function()))

The result from above is only based on a single measurement, lets look at the histogram for many runs to check whether the correct outcome has 100% probability as it should.

In [ ]:
qc = your_circuit(secret_function())
result = AerSimulator().run(qc, shots=1000, memory=True).result()
statistics = result.get_counts()
display(plot_histogram(statistics))

## 1b) Gain intuition for the circuits and the different gates

Now that we have a working implementation of Deutsch's algorithm, lets try to modify the circuit and see if it still gives the correct result. Answer the following questions as precisely as possible:

- What happens if a Hadamard gate is added or removed? Why?
- What happens if an X or Z gate is added or removed? Why?
- What happens if we add a SWAP gate?
- Does it matter if the previous changes happen before or after the query gate?
- Assuming we can only measure the other other qubit, is there a way to change the circuit to still obtain the correct result?
- If the circuit would be initialised in the Bell state $\vert \Phi^+ \rangle = \frac{1}{\sqrt{2}}\left(\vert 0 \rangle + \vert 1 \rangle \right)$ , how would you have to modify it to still obtain the correct result?

Bonus questions:
- Can you modify the `secret_function` such that you can try all four possible functions?
- Can you extend the circuit to the Deutsch-Josza algorithm for two-qubit functions?

# 2. Grovers algorithm

In this question, we want to verify that Grovers algorithm indeed works as we have predicted in the last two lectures. We will follow the lecture and check all the claims that have been made - and in the end will be able to verify the quantum advantage of this algorithm!

In [1]:
# --- Imports & helpers ---
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import DiagonalGate
from qiskit.quantum_info import Operator, Statevector
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
import numpy as np
import matplotlib.pyplot as plt

## 2a) Problem setup, $Z_f$ and $Z_{OR}$

We want to create a function that is 0 on all bitstrings except for our specific goal-string which we call `marked`. Below, you can choose the number of bits and the marked bitstring.

In [6]:
# Set problem size and the marked string (you may change this)
n = 3
marked = "101"  # <- choose your marked bitstring; e.g., "101"
assert len(marked) == n, "Length of marked string must be equal to n."

def bitstr_to_index(s: str) -> int:
    """Maps a bitstring to its computational-basis index."""
    return int(s, 2)

N = 2**n
idx_star = bitstr_to_index(marked)
print(f"N={N}, marked={marked}, index={idx_star}")

N=8, marked=101, index=5


We now want to construct the phase query gate $Z_f$ for the function which is 0 on all bistrings except the marked bitstring. You can verify that the correct matrix representation for the $Z_f$-gate is diagonal with every diagonal entry equal to $1$, except for the entry corresponding to the marked element (given by index `idx_star` using lexicographical ordering).

In particular, check that $Z_f \vert x \rangle = (-1)^{f(x)} \vert x \rangle$ for all $x$.


In [3]:
# Z_f: +1 on all states except a single -1 at the marked index
diag_Zf = np.ones(N, dtype=complex)
diag_Zf[idx_star] = -1

We can construct $Z_{OR}$ using similar reasoning: all diagonal elements should be $-1$, except the element corresponding to $\vert 00...0\rangle$, which is the first.

In [5]:
# Z_or: +1 on |000...0> and -1 on all other basis states
diag_Zor = np.full(N, -1.0 + 0.0j)
diag_Zor[0] = 1.0

From this, we can now define the gates as circuit elements in Qiskit. 

In [85]:
def Zf_gate(n: int, marked_index: int) -> DiagonalGate:
    diag = np.ones(2**n, dtype=complex)
    diag[marked_index] = -1
    return DiagonalGate(diag)

def Zor_gate(n: int) -> DiagonalGate:
    diag = np.full(2**n, -1.0 + 0.0j)
    diag[0] = 1.0
    return DiagonalGate(diag)

We now want to check that these gates are correct. Your intuition would be to simply apply the gates to a computational basis state $\vert x \rangle$ and check that the outcome is either $+\vert x \rangle$ or $-\vert x \rangle$ as appropriate. 

But: these two states differ only by a global phase, therefore measurements cannot tell you which state you have! With this in mind, can you design a way to verify that the gates work correctly, only from measurement outcomes on well-chosen input states.

In [86]:
# Verify the gates using QuantumCircuit here

## 2b) Constructing the circuit

Now we have all ingredients to construct the full circuit. Complete the function `grover_operator`. 

In [ ]:
def grover_operator(n: int, marked_index: int) -> QuantumCircuit:
    qc = QuantumCircuit(n, name="G")

    # your code here

    return qc


def grover_circuit(n: int, marked_index: int, t: int) -> QuantumCircuit:
    qc = QuantumCircuit(n, n, name=f"Grover_t={t}")
    qc.h(range(n))
    G_instr = grover_operator(n, marked_index).to_instruction()
    for _ in range(t):
        qc.append(G_instr, list(range(n)))
    qc.measure(range(n), range(n))
    return qc

In [ ]:
grover_circuit(n, idx_star, 3).draw("mpl")

## 2c) Circuit depth
How often should we apply $G$ to best separate the marked bitstring from the others?

Here we have to exploit the knowledge of how many marked bitstrings there are, in this case 1.

And now lets check if everything was correct: plot the histogram of measurement results for different numbers of implementations of the Grover operator and investigate whether our theoretically optimal $t$ is indeed the best.

In [ ]:
sim = AerSimulator()
x = 10
histograms = {}

for t in range(0, x+1):
    qc = grover_circuit(n, idx_star, t)
    tqc = transpile(qc, backend=sim)
    result = sim.run(tqc, shots=4000).result()
    counts = result.get_counts()
    histograms[t] = counts

    display(plot_histogram(counts, title=f"t = {t}"))


## 2d) Bonus - Query complexity

We have now seen that Grovers algorithm indeed works for $N=3$. In the lecture it was claimed that this algorithm scales $\mathcal{O}(\sqrt{N})$ - check this claim as far as your computing resources permit.

## 2e) Bonus - $Z_f$ and $Z_{OR}$

We have seen $Z_f$ and $Z_{OR}$ in the lecture and now constructed them easily using its matrix representation. In reality though, this way of constructing a gate is not possible, we always need to construct more complicated cicuits from the elementary gates. Using Lesson 6 from [the book](https://arxiv.org/pdf/2507.11536), can you express $Z_f$ and $Z_{OR}$ using elementary gates?